In [1]:
import pandas as pd
import numpy as np
import s3fs
import os
import time

In [2]:
from geo import stco,sub
from safegraph_py_functions import safegraph_py_functions as sgpy

In [3]:
%load_ext dotenv
%dotenv
myAccessKey = os.getenv('myAccessKey')
mySecretKey = os.getenv('mySecretKey')

start = time.time()

In [4]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [5]:
cbg_nyc = pd.read_csv('../data/nyc_cbg.csv')

In [6]:
# specify the SG key and secret
fs = s3fs.S3FileSystem(profile="safegraphws", key=myAccessKey, secret=mySecretKey, client_kwargs={'endpoint_url': 'https://s3.wasabisys.com', 'region_name':'us-east-1'})

In [7]:
#test
y = '2020'
month = '04'

#define the months and days
#years = ['2019','2020']
#monthList =["01","02","03","04","05","06", "07","08"]
#dayNumList =[31, 29, 31, 30, 31, 30, 31, 31] 
dayList =["01","02","03","04","05","06","07","08","09","10","11","12","13","14","15","16","17","18","19","20","21","22","23","24","25","26","27","28","29","30","31"]

In [8]:
##April 2020 test 1st to 3rd - to view data structure
frames = []
for i in range(0,3):
    with fs.open(f'sg-c19-response/social-distancing/v2/{y}/{month}/{dayList[i]}/{y}-{month}-{dayList[i]}-social-distancing.csv.gz','rb') as f:
        print(f'{month}-{dayList[i]}')
        # read SG's file
        df = pd.read_csv(f, escapechar='\\', compression='gzip')
        # filter NYC's Origin CBGs
        dff = pd.merge(cbg_nyc, df, left_on="orig_cbg", right_on="origin_census_block_group", how="inner")
        frames.append(dff)

04-01
04-02
04-03


In [9]:
df = pd.concat(frames)
df.head()

,Pop_10E,orig_st,orig_co,orig_cbg,orig_stco,origin_census_block_group,date_range_start,date_range_end,device_count,distance_traveled_from_home,bucketed_distance_traveled,median_dwell_at_bucketed_distance_traveled,completely_home_device_count,median_home_dwell_time,bucketed_home_dwell_time,at_home_by_each_hour,part_time_work_behavior_devices,full_time_work_behavior_devices,destination_cbgs,delivery_behavior_devices,median_non_home_dwell_time,candidate_device_count,bucketed_away_from_home_time,median_percentage_time_home,bucketed_percentage_time_home
0,11091,36,5,360050001001,36005,360050001001,2020-04-01T00:00:00-04:00,2020-04-02T00:00:00-04:00,10,14342.0,"{""16001-50000"":1,"">50000"":1,""2001-8000"":1,""1-1...","{""16001-50000"":171,"">50000"":582,""<1000"":234,""2...",1,0,"{""<60"":6,""61-360"":1}","[0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,1,1,0,0,0,0,0]",1,1,"{""360870106011"":1,""360810031002"":1,""3610319040...",1,304,40,"{""481-540"":1,""1201-1320"":1,""301-360"":1,""61-120...",0,"{""0-25"":6,""51-75"":1}"
1,1120,36,5,360050002001,36005,360050002001,2020-04-01T00:00:00-04:00,2020-04-02T00:00:00-04:00,48,6235.0,"{""0"":28,"">50000"":3,""2001-8000"":1,""1-1000"":5,""1...","{"">50000"":11,""<1000"":15,""2001-8000"":12,""1001-2...",26,1009,"{""721-1080"":12,""361-720"":2,""61-360"":4,""<60"":8,...","[26,29,30,28,29,25,26,29,30,32,30,30,31,28,26,...",1,1,"{""361190031002"":1,""360050044004"":2,""3600500420...",1,0,82,"{""21-45"":4,""46-60"":1,""721-840"":1,""<20"":30,""61-...",100,"{""26-50"":1,""51-75"":3,""0-25"":11,"">100"":4,""76-10..."
2,1974,36,5,360050002002,36005,360050002002,2020-04-01T00:00:00-04:00,2020-04-02T00:00:00-04:00,82,3437.0,"{""16001-50000"":1,""0"":43,""2001-8000"":14,""1-1000...","{""16001-50000"":5,""<1000"":92,""2001-8000"":155,""1...",43,1003,"{""721-1080"":13,""361-720"":5,""61-360"":9,""<60"":10...","[52,52,54,55,53,52,54,51,50,50,45,47,46,45,44,...",1,1,"{""360050173004"":1,""340030322023"":1,""3600502010...",1,0,189,"{""21-45"":4,""481-540"":1,""541-600"":1,""46-60"":2,""...",100,"{""26-50"":3,""51-75"":5,""0-25"":8,"">100"":2,""76-100..."
3,1240,36,5,360050002003,36005,360050002003,2020-04-01T00:00:00-04:00,2020-04-02T00:00:00-04:00,57,2320.0,"{""16001-50000"":1,""0"":27,"">50000"":1,""2001-8000""...","{""16001-50000"":218,"">50000"":47,""<1000"":300,""20...",29,927,"{""721-1080"":5,""361-720"":1,""61-360"":10,""<60"":9,...","[32,35,34,37,35,35,33,31,30,29,29,30,33,29,30,...",1,1,"{""340030234021"":1,""360610014012"":1,""3600500190...",3,0,105,"{""21-45"":1,""481-540"":1,""721-840"":1,""<20"":32,""6...",100,"{""26-50"":2,""51-75"":2,""0-25"":11,"">100"":2,""76-10..."
4,0,36,5,360050004000,36005,360050004000,2020-04-01T00:00:00-04:00,2020-04-02T00:00:00-04:00,18,2022.0,"{""0"":4,"">50000"":1,""2001-8000"":4,""1-1000"":3,""10...","{"">50000"":247,""<1000"":444,""2001-8000"":189,""100...",4,0,"{"">1080"":1,""<60"":11,""361-720"":1,""61-360"":3}","[9,4,5,4,3,2,2,1,4,1,1,1,1,3,1,2,1,1,1,2,2,1,2,1]",1,1,"{""421010390001"":1,""360050086002"":2,""3606100220...",1,182,54,"{""21-45"":2,""541-600"":2,""1201-1320"":1,""301-360""...",0,"{"">100"":1,""0-25"":9,""76-100"":2,""26-50"":1}"
